In [ ]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from sklearn.model_selection import train_test_split
import os
from PIL import Image


transform = transforms.Compose([
    transforms.Resize((480, 270)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)


cuda


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, root_dir, labels_csv, transform=None):
        self.root_dir = root_dir
        self.labels_csv = labels_csv
        self.transform = transform

        if self.labels_csv:
            labels_df = pd.read_csv(labels_csv)
            self.labels = labels_df['label']
            self.filenames = labels_df['image_name']

        else:
            self.filenames = [filename for filename in os.listdir(root_dir)]
            self.labels = [0] * len(self.filenames)

        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        filename = self.filenames[idx]
        label = self.labels[idx]

        img_path = os.path.join(self.root_dir, filename)
        image = Image.open(img_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:

from sklearn.metrics import confusion_matrix, f1_score


best_model = models.resnet18(pretrained=True).to(device)
num_ftrs = best_model.fc.in_features
best_model.fc = nn.Linear(num_ftrs, 2).to(device)
best_model.load_state_dict(torch.load('/content/drive/MyDrive/4th_Semester/Capstone/best_model.pth'))


test_original_dir = '/content/drive/MyDrive/4th_Semester/Capstone/Original_Size/val'
test_enhanced_dir = '/content/drive/MyDrive/4th_Semester/Capstone/Original_Size/val_enhanced'

test_labels_csv_path = '/content/drive/MyDrive/4th_Semester/Capstone/yolact/results/trash_val_p/comparison_results.csv'


test_original_dataset = CustomDataset(test_original_dir, test_labels_csv_path, transform=transform)
test_enhanced_dataset = CustomDataset(test_enhanced_dir, labels_csv=None, transform=transform)

test_combined_dataset = torch.utils.data.ConcatDataset([test_original_dataset, test_enhanced_dataset])


test_loader = DataLoader(test_combined_dataset, batch_size=32)



best_model.eval()
predicted_labels = []
true_labels = []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images)
        _, predicted = torch.max(outputs.data, 1)
        predicted_labels.extend(predicted.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

conf_matrix = confusion_matrix(true_labels, predicted_labels)

f1 = f1_score(true_labels, predicted_labels, average='macro')

print('Confusion Matrix:')
print(conf_matrix)
print(f'F1 Score: {f1}')


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Confusion Matrix:
[[1770  363]
 [ 121   40]]
F1 Score: 0.5107828208056625
